In [5]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
def stackImages(scale, imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range(0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape[:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]),
                                                None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y] = cv2.cvtColor(imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank] * rows
        hor_con = [imageBlank] * rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor = np.hstack(imgArray)
        ver = hor
    return ver


cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
_ , frame1 = cap.read()
_ , frame2 =cap.read()

kernel = np.ones((5,5),np.uint8)
while True : 
    fram_diff = cv2.absdiff(frame1,frame2)
    gray_image = cv2.cvtColor(fram_diff,cv2.COLOR_BGR2GRAY)
    buller_image = cv2.GaussianBlur(gray_image,(5,5),0)
    _,thres_image = cv2.threshold(buller_image,20,255,cv2.THRESH_BINARY)
    dilated_image = cv2.dilate(thres_image,kernel=kernel,iterations=3)
       

    contours, _ = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    contours ,hist = cv2.findContours(dilated_image,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours :
        x,y,w,h = cv2.boundingRect(contour)
        if cv2.contourArea(contour) <900:
            continue   
        # cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,0,255),3)
        cv2.putText(frame1,'object Moved', (20, 50), cv2.FONT_HERSHEY_SIMPLEX,  1, (0, 255, 255), 3)     

                    
    
    imgStack = stackImages(0.4,([gray_image,buller_image],
                            [thres_image,frame1]))
    cv2.imshow("Stack", imgStack)
    
    frame1 = frame2
    _,frame2  =cap.read(0)
    
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()   